An event has a set of keywords C_e

For a existing story tree S, there is a set of different C_s that is a union of all the keywords of the events inside

To know if an event is in that Story, we use compatibility between event E and story tree S with Jaccard similarity between C_s and C_e.

If we match to no stories, we create a new one

When we find a story that match, we apply either: Merge, Extend, Insert

Merge: Merge event in the story tree (merge the event with another event) (we will never have this use this)
Extend: append the event as a child node
Insert: appends the event to the root node of story tree

We extend or insert if there is no overlap of event, we find the parent event node, and measure the 'connection strength' between the new event and all the events in the story tree.  'Connection strength' is defined in page page 14 of paper.

In [21]:
import pandas as pd
import pickle
import numpy as np

In [5]:
news_dataset = pd.read_pickle("/work/IFT6010_Story_Tree/data/short_news_dataset_2_with_extractedkeyword.pickle").drop_duplicates(subset=['TEXT']).drop_duplicates(subset=['TITLE'])

In [6]:
temp_defaultdict_communities = pickle.load(open("extracted_communities.pickle",'rb'))
dict_communities = {}
for i,v in temp_defaultdict_communities.items():
    if len(v)==1:
        dict_communities[i] = v[0]

keywords_clusters = pd.DataFrame(dict_communities.items())
keywords_clusters.columns = ['keyword', 'cluster']

In [7]:
keywords_clusters['cluster'].value_counts()

0    242
1    212
2    153
3     66
4     35
5     15
Name: cluster, dtype: int64

# Define the Jaccard similarity

In [8]:
def jaccard_similarity(keyword_list_1, keyword_list_2):
    list1_as_set = set(keyword_list_1)
    intersection = list1_as_set.intersection(keyword_list_2) 

    set_union = set(keyword_list_1 + keyword_list_2)

    return len(intersection) / len(set_union)

In [ ]:
for i in news_dataset.iterrows():
    for j in news_dataset.iterrows():
        similarity = jaccard_similarity(i[1]["extracted_keywords"], j[1]["extracted_keywords"])
        if  similarity> 0.4 and similarity != 1.0:
            print(i[1]["TITLE"])
            print(j[1]["TITLE"])
            print("---------------")

How to help victims of the California wildfires
How did Northern California fires become so devastating?
---------------
How to help victims of the California wildfires
Wildfires claim at least 17 lives in Northern California
---------------
How did Northern California fires become so devastating?
How to help victims of the California wildfires
---------------
How did Northern California fires become so devastating?
More evacuations ordered as deadly wildfires scorch Northern California
---------------
How did Northern California fires become so devastating?
Wildfires claim at least 17 lives in Northern California
---------------
Are these the schools of the future?
The power of print in a digital world
---------------
Cryptic phone calls, lip-reading and a thumbs-down
John McCain's maverick moment
---------------
The power of print in a digital world
Are these the schools of the future?
---------------
John McCain's maverick moment
Cryptic phone calls, lip-reading and a thumbs-down
--

# Preprocessing and creating of vector

In [ ]:
import sys
sys.path.insert(1, '/work/IFT6010_Story_Tree/src/features/')

In [ ]:
from utils_cosine_tf_idf import latest_tfidf, preprocessing

In [ ]:
news_dataset['text_and_title'] = news_dataset[['TEXT','TITLE']].apply(lambda x :" ".join(x), axis=1)
news_dataset['preprocessed_text'] = news_dataset['text_and_title'].apply(preprocessing)


corpus = news_dataset['preprocessed_text']
corpus = corpus.to_list()

news_dataset['VECTOR'] = news_dataset['preprocessed_text'].apply(latest_tfidf, allDocs=corpus)

# Extend and insert event in story

Measure connection strength between our event X, and all the events X_s of a story. We look at:

1) The time distance between both events

2) The compatibility of the two events

3) The storyline coherence if we append event X to story tree of X_s

connection_strength(X,X_s) = compatibility()+coherence()+time_penalty

In [35]:
def compatiblity(tf_new_event, tf_event_story):

    #tf_new_event = np.array(tf_new_event)
    #tf_event_story = np.array(tf_event_story)

    if len(tf_new_event) < len(tf_event_story):
        temp = tf_new_event
        vector_a = tf_event_story
        vector_b = temp
    else:
        vector_a = tf_new_event
        vector_b = tf_event_story

    list_1={}
    list_2={}

    for elem in vector_a:
        if elem in vector_b:
            list_1[elem] = vector_a[elem]
            list_2[elem] = vector_b[elem]
        else:
            list_1[elem] = vector_a[elem]
            list_2[elem] = 0

    for elem in vector_b:
        if not elem in list_1:
            list_1[elem] = 0 
            list_2[elem] = vector_b[elem]

    # turn dictionary to numpy array
    list_1_vector = np.fromiter(list_1.values(), dtype=float)
    list_2_vector = np.fromiter(list_2.values(), dtype=float)

    prod = np.dot(list_1_vector, list_2_vector)

    
    return prod / np.linalg.norm(list_1_vector) * np.linalg.norm(list_2_vector)

In [ ]:
def conherence():
    event1 = news_dataset['VECTOR'].iloc[2]

    sum_ = 0
    
    for event in stories:
        event2 = news_dataset['VECTOR'].iloc[3]

        sum_ += compatiblity(event1, event2)

    return sum_/len(stories)

In [ ]:
def time_penalty(delta, time1, time2):
    if time1 < time2:
        return math.exp(delta)
    
    return 0

# Create stories

In [ ]:
compatiblity(news_dataset['VECTOR'].iloc[2], news_dataset['VECTOR'].iloc[3])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=902703f2-430f-48f3-ba3f-6c2fee66cf11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>